*By: B830459\
Date: 20/1/2023*

## Program Explanation: 

The program produces a database file, for which the user can modify its entries. The entries are based upon a csv file, specified by the user and its folder location. From one csv file, there can be 2 diffrent formats of tables that could be produced and inserted into the database. Those fromats have been named "Records" and "Timetable".\
The "Records" format represents a dataframe containing students attendance records for the specified module, 
whilist the "Timetable" format represents a dataframe containing the module specific events over the weeks for the module.

In order for the above to be executed the user, must specify the function **to_sql( )** parameters.\
Those are subfolder of the csv files location from the current directory as a string type and the csv file name as a string type, from which the user would like to create the entry tables for the database.\
Upon running the program, the user is promted with a 3 options. Those are with regard to whether to user would like to continue the process of inserting tables into the database and if so what format should the table take, given the csv file. After an option has been choosen, the user is asked to name the table that will be stored into the database.

In the case that an invalid formatting type has been entered out of the specified options, a "Try again" message will be returned, alongside the initial 3 options described above. 

In order to program to function as intended, the csv files that the user specifies, should have the all the module specific events and their properties in the first row of the csv file. Otherwise the "Timetable" provided formatting won't work as intended.

## Imports

In [2]:
import pandas as pd 
import sqlite3 

## Reading Files

In [3]:
def readcsv(folder, filename):
    """
    Reads csv files provided a:
    
    folder -> folder name where the csv 
    file is located in as a string type
    filename -> the filename of the csv file 
    as a string type
    
    """
    destination = folder + "/" + filename + ".csv"
    raw_file = pd.read_csv(destination)
    return raw_file

## DataFrame Preparations

In [4]:
def indexed(df):
    """
    
    Renames the first columns of the 
    dataframe as "student_id" and 
    sets it as an index.
    
    df -> the dataframe on which the
    manpilations will be executed on 
    
    """
    df.rename(columns = {df.columns[0]: "student_id"}, inplace = True)
    df = df.set_index("student_id")
    return df

In [5]:
def sessions(df):
    """
    
    Takes the columns names of the 
    provided dataframe and returns 
    another  dataframe, based on the  
    column parameters. 
    As those would be including information 
    about the individual sessions 
    across a module.
    
    df -> the dataframe on which the
    manpilations will be executed on
    
    """
    table_contents = []
    for i in range(indexed(df).shape[1]):
        x = indexed(df).columns[i].split("\n")
        x.pop()
        table_contents.append(x)
    
    df_sessions = pd.DataFrame(table_contents)
    
    # separate the Week and Semester entries into a 2 seperate ones
    df_sessions[["s", "w"]] = df_sessions[0].str.split(".", 1, expand = True)
    df_sessions = df_sessions.drop(0, axis = 1)
    
    session_id = []
    for i in range(df_sessions.shape[0]):
        session_id.append(i)
    
    df_sessions["session_id"] = session_id
    
    # rename and reorder the columns in desired order
    df_sessions.columns = ["date", "time", "type", "room", "sem", "week", "sess_id"]
    df_sessions_clean = df_sessions[["sess_id", "sem", "week", "date", "time", "type", "room",]]
    
    df_sessions_clean.set_index("sess_id", inplace = True)
    
    return df_sessions_clean

In [6]:
def cleaned(df):
    """
    
    Takes a dataframe with student
    attendance records and alters it 
    in the following way:
    
    GPS -> Boolean True
    X -> Boolean False
    Ex -> Null value
    
    Further all the rows filled 
    entirly with Nan's are dropped.
    
    """
    
    # give short names with unique ID
    col_names = []
    clean = indexed(df)
    for i in range(clean.shape[1]):
        x = clean.columns[i].split("\n")
        x = x[0] + " Session ID: " + str(i)
        col_names.append(x)
        
    clean.columns = col_names
    
    # rename attendance cases
        # I had issues with this code, when ran the provided environment
        # they were not saving the null values as supposed, but rather 
        # as trues. I leave them just in case here for referance. 
        #clean.replace(to_replace = "GPS", value = True, inplace = True)
        #clean.replace(to_replace = "X", value = False, inplace = True)
        #clean.replace(to_replace = "Ex", value = None, inplace = True)
    clean.replace({'Ex' : None}, inplace = True)
    clean.replace({'GPS': True}, inplace = True)
    clean.replace({'X' : False}, inplace = True)
    
    # remove Nan's column and row wise 
    clean = clean.dropna(axis = 0, how = "all")
    clean = clean.dropna(axis = 1, how = "all")
    
    # Add _ for the blank spaces in column names to prevent
    # any potential errors
    clean.columns = clean.columns.str.replace(" ", "_")
    
    return clean

## Main

In [7]:
def tosql(folder, filename):
    """
    
    Given a csv filename and its folder
    location the function
    propmpts questions with regards
    of what type manipulations to carry out 
    stores the transformed dataframe
    to the database as a table. 
    
    
    folder -> folder name where the csv 
    file is located in as a string type
    filename -> the filename of the csv file 
    as a string type
    
    """
    conn = sqlite3.connect("CWDatabase.db")
    df = readcsv(folder, filename)
    while True:
        a = str(input("What type of table would you want to import?: (r) for records / (t) for timetable/ (q) for exit: "))
        if a.lower() == 'q':
            print("Bye!")
            break
        elif a.strip().lower() == "r" or a.strip().lower() == "t":
            b = str(input("How would you want to name the table?:"))
            if a.strip().lower() == "r":
                cleaned(df).to_sql(b, conn, if_exists = "replace", index = True)
            else: 
                sessions(df).to_sql(b, conn, if_exists = "replace", index = True)
        else:   
                print("Invalid input")
                print("Try again or enter 'q' to exit")
        
    conn.close()

## Outputs

In [8]:
tosql("Attendance", "22COA111ModuleRegister")

What type of table would you want to import?: (r) for records / (t) for timetable/ (q) for exit: R
How would you want to name the table?:dfcoa111
What type of table would you want to import?: (r) for records / (t) for timetable/ (q) for exit: T
How would you want to name the table?:sesscoa111
What type of table would you want to import?: (r) for records / (t) for timetable/ (q) for exit: Q
Bye!


In [9]:
tosql("Attendance", "22COA122ModuleRegister")

What type of table would you want to import?: (r) for records / (t) for timetable/ (q) for exit: R
How would you want to name the table?:dfcoa122
What type of table would you want to import?: (r) for records / (t) for timetable/ (q) for exit: T
How would you want to name the table?:sesscoa122
What type of table would you want to import?: (r) for records / (t) for timetable/ (q) for exit: Q
Bye!


## Testing 

In [11]:
coa111 = readcsv("Attendance", "22COA111ModuleRegister")
coa111.head()

,sid_x,S1.W1\nMonday (03-10-2022)\n09:00 - 10:00\nLecture\nU020\n,S1.W1\nWednesday (05-10-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W2\nWednesday (12-10-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W2\nWednesday (12-10-2022)\n12:00 - 13:00\nTutorial\n\n,S1.W2\nThursday (13-10-2022)\n14:00 - 15:00\nLecture\nCC011\n,S1.W3\nWednesday (19-10-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W3\nWednesday (19-10-2022)\n12:00 - 13:00\nTutorial\n\n,S1.W3\nThursday (20-10-2022)\n14:00 - 15:00\nLecture\nCC011\n,S1.W4\nWednesday (26-10-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W4\nWednesday (26-10-2022)\n12:00 - 13:00\nTutorial\n\n,S1.W4\nThursday (27-10-2022)\n14:00 - 15:00\nLecture\nCC011\n,S1.W5\nWednesday (02-11-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W5\nWednesday (02-11-2022)\n12:00 - 13:00\nTutorial\n\n,S1.W5\nThursday (03-11-2022)\n14:00 - 15:00\nLecture\nCC011\n
0,0,GPS,GPS,X,NaN,GPS,GPS,NaN,GPS,GPS,NaN,X,X,NaN,X
1,1,GPS,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS
2,2,GPS,GPS,GPS,NaN,X,X,NaN,GPS,X,NaN,GPS,X,NaN,X
3,3,NaN,NaN,GPS,NaN,GPS,X,NaN,GPS,X,NaN,GPS,X,NaN,GPS
4,4,GPS,X,X,NaN,GPS,X,NaN,GPS,X,NaN,GPS,X,NaN,GPS


In [12]:
indexed(coa111).head()

,S1.W1\nMonday (03-10-2022)\n09:00 - 10:00\nLecture\nU020\n,S1.W1\nWednesday (05-10-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W2\nWednesday (12-10-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W2\nWednesday (12-10-2022)\n12:00 - 13:00\nTutorial\n\n,S1.W2\nThursday (13-10-2022)\n14:00 - 15:00\nLecture\nCC011\n,S1.W3\nWednesday (19-10-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W3\nWednesday (19-10-2022)\n12:00 - 13:00\nTutorial\n\n,S1.W3\nThursday (20-10-2022)\n14:00 - 15:00\nLecture\nCC011\n,S1.W4\nWednesday (26-10-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W4\nWednesday (26-10-2022)\n12:00 - 13:00\nTutorial\n\n,S1.W4\nThursday (27-10-2022)\n14:00 - 15:00\nLecture\nCC011\n,S1.W5\nWednesday (02-11-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W5\nWednesday (02-11-2022)\n12:00 - 13:00\nTutorial\n\n,S1.W5\nThursday (03-11-2022)\n14:00 - 15:00\nLecture\nCC011\n
student_id,,,,,,,,,,,,,,
0,GPS,GPS,X,NaN,GPS,GPS,NaN,GPS,GPS,NaN,X,X,NaN,X
1,GPS,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS
2,GPS,GPS,GPS,NaN,X,X,NaN,GPS,X,NaN,GPS,X,NaN,X
3,NaN,NaN,GPS,NaN,GPS,X,NaN,GPS,X,NaN,GPS,X,NaN,GPS
4,GPS,X,X,NaN,GPS,X,NaN,GPS,X,NaN,GPS,X,NaN,GPS


In [13]:
cleaned(coa111).head()

,S1.W1_Session_ID:_0,S1.W1_Session_ID:_1,S1.W2_Session_ID:_2,S1.W2_Session_ID:_4,S1.W3_Session_ID:_5,S1.W3_Session_ID:_7,S1.W4_Session_ID:_8,S1.W4_Session_ID:_10,S1.W5_Session_ID:_11,S1.W5_Session_ID:_13
student_id,,,,,,,,,,
0,True,True,False,True,True,True,True,False,False,False
1,True,True,True,True,True,True,True,True,True,True
2,True,True,True,False,False,True,False,True,False,False
3,NaN,NaN,True,True,False,True,False,True,False,True
4,True,False,False,True,False,True,False,True,False,True


Testing whether the null values are carried out as supposed

In [14]:
cleaned(coa111).loc[81]

S1.W1_Session_ID:_0     False
S1.W1_Session_ID:_1     False
S1.W2_Session_ID:_2      None
S1.W2_Session_ID:_4      None
S1.W3_Session_ID:_5     False
S1.W3_Session_ID:_7      True
S1.W4_Session_ID:_8      True
S1.W4_Session_ID:_10     True
S1.W5_Session_ID:_11     True
S1.W5_Session_ID:_13    False
Name: 81, dtype: object

In [15]:
sessions(coa111).head()

,sem,week,date,time,type,room
sess_id,,,,,,
0,S1,W1,Monday (03-10-2022),09:00 - 10:00,Lecture,U020
1,S1,W1,Wednesday (05-10-2022),10:00 - 11:00,Lecture,SMB014
2,S1,W2,Wednesday (12-10-2022),10:00 - 11:00,Lecture,SMB014
3,S1,W2,Wednesday (12-10-2022),12:00 - 13:00,Tutorial,
4,S1,W2,Thursday (13-10-2022),14:00 - 15:00,Lecture,CC011
